# Constrained Tikhonov regularization implemented by (accelerated) forward-backward splitting
We consider the two-dimensional deconvolution problems to find a non-negative function f given data 
$$
    d \sim \mathrm{Pois}(h*f)
$$
with a non-negative convolution kernel $h$, and $\mathrm{Pois}$ denotes the element-wise Poisson distribution.

We explore the use of the semismooth Newton method to implement constrained Tikhonov regularization 
$$
\hat{f} = \mathrm{argmin}_{f\geq 0} \left[\| h*f-d\|^2_{L^2(w)} + \alpha \|f\|^2_{L^2}\right]
$$
with a weight $w = \frac{1}{\sqrt{d+1}}$. The regularization parameter $\alpha$ is chosen by the discrepancy principle.   

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mplib
from regpy.operators import CoordinateMask 
from regpy.operators.convolution import ConvolutionOperator, GaussianBlur, ExponentialConvolution
from regpy.vecsps import UniformGridFcts
from regpy.solvers import TikhonovRegularizationSetting, RegularizationSetting
from regpy.solvers.linear.semismoothNewton import SemismoothNewton_bilateral, SemismoothNewton_nonneg, SemismoothNewtonAlphaGrid
from regpy.solvers.linear.proximal_gradient import ForwardBackwardSplitting, FISTA
from regpy.solvers.linear.primal_dual import PDHG
from regpy.hilbert import L2, HmDomain
from regpy.stoprules import CountIterations, Discrepancy, DualityGapStopping
from regpy.functionals import QuadraticLowerBound, QuadraticBilateralConstraints
import logging
from numpy.linalg import norm

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(name)-20s :: %(message)s'
)

### plotting routine for comparing reconstructions and originals 

In [ ]:
def comparison_plot(grid,truth,reco,title_right='exact',title_left='reconstruction',residual=None):
    plt.rcParams.update({'font.size': 22})
    extent = [grid.axes[0][0],grid.axes[0][-1], grid.axes[1][0], grid.axes[1][-1]]
    maxval = np.max(truth[:]); minval = np.min(truth[:])
    mycmap = mplib.colormaps['hot']
    mycmap.set_over((0,0,1.,1.))  # use blue as access color for large values
    mycmap.set_under((0,1,0,1.))  # use green as access color for small values
    if not (residual is None):
        fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize = (22,16))
    else:
        fig, (ax1,ax2) = plt.subplots(1,2,figsize = (22,8))
    im1= ax1.imshow(reco.T,extent=extent,origin='lower',
                    vmin=1.05*minval-0.05*maxval, vmax =1.05*maxval-0.05*minval,
                    cmap=mycmap
                    )
    ax1.title.set_text(title_left)
    fig.colorbar(im1,extend='both')
    im2= ax2.imshow(truth.T,extent=extent, origin='lower',
                    vmin=1.05*minval-0.05*maxval, vmax =1.05*maxval-0.05*minval,
                    cmap=mycmap
                    )
    ax2.title.set_text(title_right)
    fig.colorbar(im2,extend='both',orientation='vertical')
    if not (residual is None):
        maxv = np.max(reco[:]-truth[:])
        im3 = ax3.imshow(truth.T-reco.T,extent=extent, origin='lower',vmin= -maxv,vmax=maxv,cmap='RdYlBu_r')
        ax3.title.set_text('reconstruction error')
        fig.colorbar(im3)

        maxv = np.max(residual[:])
        im4 = ax4.imshow(residual.T,extent=extent, origin='lower',vmin= -maxv,vmax=maxv,cmap='RdYlBu_r')
        ax4.title.set_text('data residual')
        fig.colorbar(im4)

### test objects

In [ ]:
grid = UniformGridFcts((-1, 1, 256), (-1.5, 1, 256),dtype = float, periodic = True)
"""Space of real-valued functions on a uniform grid with rectangular pixels"""
X = grid.coords[0]; Y = grid.coords[1]
"""x and y coordinates."""
cross = 1.0*np.logical_or((abs(X)<0.01) * (abs(Y)<0.3),(abs(X)<0.3) * (abs(Y)<0.01)) 
rad = np.sqrt(X**2 + Y**2)
ring = 1.0*np.logical_and(rad>=0.9, rad<=0.95)
smallbox = (abs(X+0.55)<=0.05) * (abs(Y-0.55)<=0.05)
bubbles = (1.001+np.sin(50/(X+1.3)))*np.exp(-((Y+1.25)/0.1)**2)*(X>-0.8)*(X<0.8)

ramp = Y<=-1

objects = 200*(ring + 2.0*cross + 1.5*smallbox + 2*ramp -bubbles)
exact_sol = objects 


### creating Poisson distributed synthetic data

In [ ]:
a=0.15
conv =  GaussianBlur(grid,a,pad_amount=((16,16),(16,16)))
"""Convolution operator \(f\mapsto h*f\) for the convolution kernel \(h(x)=\exp(-|x|_2^2/a^2)\)."""
blur = conv(exact_sol)
blur[blur<0] = 0.
"""Simulated exact data."""
data = np.random.poisson(blur)
"""Simulated measured data. The Poisson distribution occurs if photon count detectors are used."""
comparison_plot(grid,exact_sol,data,title_left='noisy measurement data')

In [ ]:
weighted_data_space = L2(grid, weights = 1./(1.+data))
setting = RegularizationSetting(op=conv, penalty=L2, data_fid=weighted_data_space)
alpha = 0.001

SSNewton_nn = SemismoothNewton_nonneg(setting, data, alpha, TOL = 0.01, 
                                    logging_level=logging.DEBUG, cg_logging_level=logging.INFO)
it = iter(SSNewton_nn)

# Reconstructions with one sided constraints

In [ ]:
penLower = QuadraticLowerBound(grid,x0=0,lb=0)
alpha = 1e-3
n_iter = 400 
settingLower = TikhonovRegularizationSetting(op=conv, penalty=penLower, data_fid = L2,data_fid_shift=data,regpar=alpha)

### Forward-backward splitting

In [ ]:
FB_solver_lb = ForwardBackwardSplitting(settingLower)
stop_FB_lb=DualityGapStopping(FB_solver_lb,threshold = 1., max_iter=n_iter,logging_level=logging.WARNING)
FB_solver_lb.run(stoprule=stop_FB_lb)

comparison_plot(grid,exact_sol,FB_solver_lb.x,title_left='Forward-backward')

### FISTA

In [ ]:
FISTA_solver_lb = FISTA(settingLower)
stop_FISTA_lb=DualityGapStopping(FISTA_solver_lb,threshold = 1., max_iter=n_iter,logging_level=logging.INFO)
FISTA_solver_lb.run(stoprule=stop_FISTA_lb)
    
comparison_plot(grid,exact_sol,FISTA_solver_lb.x,title_left='FISTA reco')

In [ ]:
PDHG_solver_lb = PDHG(settingLower)
stop_PDHG_lb=DualityGapStopping(PDHG_solver_lb,threshold = 1., max_iter=n_iter,logging_level=logging.INFO)
PDHG_solver_lb.run(stoprule=stop_PDHG_lb)

comparison_plot(grid,exact_sol,PDHG_solver_lb.x, title_left='PDHG reco')

In [ ]:
plt.semilogy(stop_FB_lb.gap_stat,label='ForwardBackward')
plt.semilogy(stop_FISTA_lb.gap_stat,label='FISTA')
plt.semilogy(stop_PDHG_lb.gap_stat,label='PDHG')
plt.legend()
plt.xlabel('it. step'); plt.ylabel('duality gap')
plt.title('convergence for nonnegativity constraint')

# Reconstructions for bilateral constraints

### define setting

In [ ]:
pen = QuadraticBilateralConstraints(grid,lb=0, ub=400,eps=1e-14)
alpha = 1e-4
n_iter = 1000 
settingBilateral = TikhonovRegularizationSetting(op=conv, penalty=pen, data_fid = weighted_data_space,data_fid_shift=data,regpar=alpha)

### Forward-backward splitting

In [ ]:
FB_solver_bil = ForwardBackwardSplitting(settingBilateral)
stop_FB_bil = DualityGapStopping(FB_solver_bil,threshold = 1., max_iter=n_iter,logging_level=logging.INFO)
FB_solver_bil.run(stoprule=stop_FB_bil)

comparison_plot(grid,exact_sol,FB_solver_bil.x,title_left='Forward-backward')

In [ ]:
FISTA_solver_bil = FISTA(settingBilateral)
stop_FISTA_bil = DualityGapStopping(FISTA_solver_bil,threshold = 1., max_iter=n_iter,logging_level=logging.INFO)
gap_FISTA_bil = np.zeros((stop_FISTA_bil.max_iter+1,))
gap_FISTA_bil[0] = FISTA_solver_bil.gap
for step_nr, (x,y)  in enumerate(FISTA_solver_bil.while_(stop_FISTA_bil)):
    gap_FISTA_bil[step_nr+1] = FISTA_solver_bil.gap
comparison_plot(grid,exact_sol,x,title_left='FISTA reco')

### Primal-dual hybrid gradient (Chambolle-Pock) methods

In [ ]:
PDHG_solver_bil = PDHG(settingBilateral)
stop_PDHG_bil = DualityGapStopping(PDHG_solver_bil,threshold = 1., max_iter=n_iter,logging_level=logging.INFO)
PDHG_solver_bil.run(stoprule=stop_PDHG_bil)

comparison_plot(grid,exact_sol,PDHG_solver_bil.x,title_left='PDHG reco')

In [ ]:
plt.semilogy(stop_FB_bil.gap_stat,label='ForwardBackward')
plt.semilogy(stop_FISTA_bil.gap_stat,label='FISTA')
plt.semilogy(stop_PDHG_bil.gap_stat,label='PDHG')
plt.legend()
plt.xlabel('it. step'); plt.ylabel('duality gap')
plt.title('convergence for bilateral constraints')